# IMPORT FUNCFIONS

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import itertools
import pickle
import os
import random
from sklearn.model_selection import train_test_split

from sklearn.metrics import average_precision_score
from sklearn import metrics

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from ge import LINE

# READ DATA

In [2]:
def get_edges_dict(traindata):
    if os.path.isfile("edgesdic.pkl"):
        f = open("edgesdic.pkl","rb")
        edges = pickle.load(f)
        f.close()
        return edges
    else:
        edges = dict()
        for edge in range(len(traindata)): 
            edges[(traindata.loc[[edge]]['id_1'].values[0],traindata.loc[[edge]]['id_2'].values[0])] = 1 
            edges[(traindata.loc[[edge]]['id_2'].values[0],traindata.loc[[edge]]['id_1'].values[0])] = 1  
        f = open("edgesdic.pkl","wb")
        pickle.dump(edges,f)
        f.close()
        return edges

def get_negative_edges(traindata,g):
    edges = get_edges_dict(traindata)
    # for missing edges.
    negative_edges = set([])
    maxNodenum = max(max(traindata['id_1']),max(traindata['id_2']))
    #產生與原圖同edge數目的negative edges
    while (len(negative_edges)<len(traindata)):
        node1=random.randint(0, maxNodenum) 
        node2=random.randint(0, maxNodenum) 
        tmp = edges.get((node1,node2),0) #edge不在graph裡標為0
        if tmp == 0 and node1!=node2 and g.has_node(node1) and g.has_node(node2): # if edge不在graph裡才要做處理
            negative_edges.add((node1,node2))
        else:
            continue
    return negative_edges

# MODELS

In [3]:
def combine_embedding(data,embeddings):
    i=0
    X = []
    for node1,node2 in data:
        X.append(np.concatenate((data[i],embeddings[int(node1)],embeddings[int(node2)])))
   # print(embeddings[str(int(data[0]))])
        i+=1
    return X

In [4]:
all_ap = []
all_auc = []

filename = 'polbooks'

for i in range(10):
    ori_df = pd.read_csv('data/'+filename+'.txt', header=None, sep=' ')
    ori_df.columns = ['id_1', 'id_2']

    ori_G = nx.from_pandas_edgelist(ori_df, 'id_1', 'id_2')
    #print(nx.info(ori_G))
    
    negative_edges = get_negative_edges(ori_df, ori_G)
    #print(len(negative_edges))

    train_df, test_df = train_test_split(ori_df, test_size=0.1)
    
    nodes_number = len(ori_G.nodes)
    graph_np = np.zeros((nodes_number, nodes_number))
    for i in range(train_df.shape[0]):
        graph_np[train_df.iloc[i, 0], train_df.iloc[i, 1]] = 1
        graph_np[train_df.iloc[i, 1], train_df.iloc[i, 0]] = 1

    train_G = nx.from_numpy_matrix(graph_np)
    #print(nx.info(train_G))
    
    df_neg = pd.DataFrame(list(negative_edges), columns=['id_1', 'id_2'])
    
    train_df = train_df.values
    test_pos_df = test_df.values
    df_neg = df_neg.values
    
    model = LINE(train_G, embedding_size=12, order='all')
    model.train(batch_size=128, epochs=50, verbose=2)
    
    LINE_embeddings = model.get_embeddings()
    
    train_df = combine_embedding(train_df, LINE_embeddings)
    test_pos_df = combine_embedding(test_pos_df, LINE_embeddings)
    df_neg = combine_embedding(df_neg, LINE_embeddings)
    
    X_train_neg, X_test_neg = train_test_split(df_neg, test_size=0.1)
    
    y_train_pos = np.ones(len(train_df))
    y_train_neg = np.zeros(len(X_train_neg))
    y_test_pos = np.ones(len(test_pos_df))
    y_test_neg = np.zeros(len(X_test_neg))
    
    X_train = np.concatenate((train_df, X_train_neg))
    y_train = np.concatenate((y_train_pos, y_train_neg))
    X_test = np.concatenate((test_pos_df, X_test_neg))
    y_test = np.concatenate((y_test_pos, y_test_neg))
    
    clf = RandomForestClassifier(n_estimators=400)
    clf.fit(X_train, y_train)
    
    predict_Y = clf.predict(X_test)
    
    ap = average_precision_score(y_test, predict_Y)
    print("AP： ", ap)
    fpr, tpr, _ = metrics.roc_curve(y_test, predict_Y, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    print("AUC SCORE: ",auc)
    
    all_ap.append(ap)
    all_auc.append(auc)

print("####################### SUM UP ############################")
print("AP MEAN : ", np.array(all_ap).mean())
print("AP STD : ", np.array(all_ap).std())
print("AUC MEAN : ", np.array(all_auc).mean())
print("AUC STD : ", np.array(all_auc).std())

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/50
19/19 - 0s - loss: 1.3863 - first_order_loss: 0.6932 - second_order_loss: 0.6931
Epoch 2/50
19/19 - 0s - loss: 1.3860 - first_order_loss: 0.6931 - second_order_loss: 0.6928
Epoch 3/50
19/19 - 0s - loss: 1.3854 - first_order_loss: 0.6930 - second_order_loss: 0.6924
Epoch 4/50
19/19 - 0s - loss: 1.3850 - first_order_loss: 0.6932 - second_order_loss: 0.6919
Epoch 5/50
19/19 - 0s - loss: 1.3837 - first_order_loss: 0.6931 - second_order_loss: 0.6906
Epoch 6/50
19/19 - 0s - loss: 1.3812 - first_order_loss: 0.6930 - second_order_loss: 0.6882
Epoch 7/50
19/19 - 0s - loss: 1.3786 - first_order_loss: 0.6929 - second_order_loss: 0.6858
Epoch 8/50
19/19 - 0s - loss: 1.3737 - first_order_loss: 0.6928 - second_order_loss:

Epoch 27/50
19/19 - 0s - loss: 1.1119 - first_order_loss: 0.6902 - second_order_loss: 0.4223
Epoch 28/50
19/19 - 0s - loss: 1.0766 - first_order_loss: 0.6891 - second_order_loss: 0.3868
Epoch 29/50
19/19 - 0s - loss: 1.0893 - first_order_loss: 0.6903 - second_order_loss: 0.3995
Epoch 30/50
19/19 - 0s - loss: 1.0710 - first_order_loss: 0.6884 - second_order_loss: 0.3824
Epoch 31/50
19/19 - 0s - loss: 1.0894 - first_order_loss: 0.6860 - second_order_loss: 0.4026
Epoch 32/50
19/19 - 0s - loss: 1.0561 - first_order_loss: 0.6890 - second_order_loss: 0.3674
Epoch 33/50
19/19 - 0s - loss: 1.0537 - first_order_loss: 0.6886 - second_order_loss: 0.3651
Epoch 34/50
19/19 - 0s - loss: 1.0423 - first_order_loss: 0.6880 - second_order_loss: 0.3551
Epoch 35/50
19/19 - 0s - loss: 1.0446 - first_order_loss: 0.6862 - second_order_loss: 0.3580
Epoch 36/50
19/19 - 0s - loss: 1.0420 - first_order_loss: 0.6861 - second_order_loss: 0.3562
Epoch 37/50
19/19 - 0s - loss: 1.0717 - first_order_loss: 0.6844 - sec

Epoch 15/50
19/19 - 0s - loss: 1.3106 - first_order_loss: 0.6924 - second_order_loss: 0.6183
Epoch 16/50
19/19 - 0s - loss: 1.2897 - first_order_loss: 0.6922 - second_order_loss: 0.5972
Epoch 17/50
19/19 - 0s - loss: 1.2736 - first_order_loss: 0.6923 - second_order_loss: 0.5816
Epoch 18/50
19/19 - 0s - loss: 1.2521 - first_order_loss: 0.6920 - second_order_loss: 0.5600
Epoch 19/50
19/19 - 0s - loss: 1.2512 - first_order_loss: 0.6918 - second_order_loss: 0.5592
Epoch 20/50
19/19 - 0s - loss: 1.2105 - first_order_loss: 0.6918 - second_order_loss: 0.5191
Epoch 21/50
19/19 - 0s - loss: 1.2022 - first_order_loss: 0.6914 - second_order_loss: 0.5105
Epoch 22/50
19/19 - 0s - loss: 1.1708 - first_order_loss: 0.6915 - second_order_loss: 0.4791
Epoch 23/50
19/19 - 0s - loss: 1.1660 - first_order_loss: 0.6914 - second_order_loss: 0.4750
Epoch 24/50
19/19 - 0s - loss: 1.1319 - first_order_loss: 0.6912 - second_order_loss: 0.4401
Epoch 25/50
19/19 - 0s - loss: 1.1440 - first_order_loss: 0.6896 - sec

Epoch 3/50
19/19 - 0s - loss: 1.3854 - first_order_loss: 0.6930 - second_order_loss: 0.6924
Epoch 4/50
19/19 - 0s - loss: 1.3847 - first_order_loss: 0.6930 - second_order_loss: 0.6917
Epoch 5/50
19/19 - 0s - loss: 1.3835 - first_order_loss: 0.6932 - second_order_loss: 0.6904
Epoch 6/50
19/19 - 0s - loss: 1.3809 - first_order_loss: 0.6930 - second_order_loss: 0.6879
Epoch 7/50
19/19 - 0s - loss: 1.3772 - first_order_loss: 0.6928 - second_order_loss: 0.6843
Epoch 8/50
19/19 - 0s - loss: 1.3722 - first_order_loss: 0.6929 - second_order_loss: 0.6793
Epoch 9/50
19/19 - 0s - loss: 1.3640 - first_order_loss: 0.6929 - second_order_loss: 0.6711
Epoch 10/50
19/19 - 0s - loss: 1.3547 - first_order_loss: 0.6926 - second_order_loss: 0.6620
Epoch 11/50
19/19 - 0s - loss: 1.3429 - first_order_loss: 0.6926 - second_order_loss: 0.6503
Epoch 12/50
19/19 - 0s - loss: 1.3293 - first_order_loss: 0.6927 - second_order_loss: 0.6367
Epoch 13/50
19/19 - 0s - loss: 1.3238 - first_order_loss: 0.6925 - second_ord

Epoch 41/50
19/19 - 0s - loss: 1.0229 - first_order_loss: 0.6848 - second_order_loss: 0.3385
Epoch 42/50
19/19 - 0s - loss: 1.0177 - first_order_loss: 0.6844 - second_order_loss: 0.3334
Epoch 43/50
19/19 - 0s - loss: 1.0459 - first_order_loss: 0.6819 - second_order_loss: 0.3624
Epoch 44/50
19/19 - 0s - loss: 1.0038 - first_order_loss: 0.6827 - second_order_loss: 0.3222
Epoch 45/50
19/19 - 0s - loss: 1.0107 - first_order_loss: 0.6807 - second_order_loss: 0.3285
Epoch 46/50
19/19 - 0s - loss: 0.9883 - first_order_loss: 0.6799 - second_order_loss: 0.3078
Epoch 47/50
19/19 - 0s - loss: 0.9763 - first_order_loss: 0.6832 - second_order_loss: 0.2922
Epoch 48/50
19/19 - 0s - loss: 1.0157 - first_order_loss: 0.6798 - second_order_loss: 0.3366
Epoch 49/50
19/19 - 0s - loss: 1.0256 - first_order_loss: 0.6753 - second_order_loss: 0.3514
Epoch 50/50
19/19 - 0s - loss: 1.0066 - first_order_loss: 0.6817 - second_order_loss: 0.3231
AP：  0.6375
AUC SCORE:  0.7000000000000001
Epoch 1/50
19/19 - 0s - los

Epoch 29/50
19/19 - 0s - loss: 1.0839 - first_order_loss: 0.6900 - second_order_loss: 0.3944
Epoch 30/50
19/19 - 0s - loss: 1.0786 - first_order_loss: 0.6898 - second_order_loss: 0.3887
Epoch 31/50
19/19 - 0s - loss: 1.1221 - first_order_loss: 0.6884 - second_order_loss: 0.4338
Epoch 32/50
19/19 - 0s - loss: 1.0586 - first_order_loss: 0.6897 - second_order_loss: 0.3689
Epoch 33/50
19/19 - 0s - loss: 1.0607 - first_order_loss: 0.6884 - second_order_loss: 0.3726
Epoch 34/50
19/19 - 0s - loss: 1.0486 - first_order_loss: 0.6882 - second_order_loss: 0.3611
Epoch 35/50
19/19 - 0s - loss: 1.0436 - first_order_loss: 0.6883 - second_order_loss: 0.3546
Epoch 36/50
19/19 - 0s - loss: 1.0408 - first_order_loss: 0.6894 - second_order_loss: 0.3518
Epoch 37/50
19/19 - 0s - loss: 1.0762 - first_order_loss: 0.6860 - second_order_loss: 0.3892
Epoch 38/50
19/19 - 0s - loss: 1.0141 - first_order_loss: 0.6845 - second_order_loss: 0.3291
Epoch 39/50
19/19 - 0s - loss: 1.0379 - first_order_loss: 0.6867 - sec